In [1]:
%load_ext autoreload
%autoreload 2

In [39]:
def batches(r, y):
    return r//y + 1*(r%y != 0)

class stockpile:
    def __init__(self, reagents):
        self.__total_ore_cost = 0
        self.element_requirements = {}
        self.reagent_dict = reagents
        
        
    def order(self, reagent_name, amount):
        reagent_obj = self.reagent_dict[reagent_name]
        
        if reagent_obj.is_element:
            if reagent_obj.name in self.element_requirements:
                self.element_requirements[reagent_obj.name] += amount
            else:
                self.element_requirements[reagent_obj.name] = amount
        else:
            for r_name, r_cost in reagent_obj.costs.items():
                self.order(r_name, amount*r_cost)
        
    def make_elements(self):
        for r_name, r_req in self.element_requirements.items():
            reagent_obj = self.reagent_dict[r_name]
            n_batches = batches(r_req, reagent_obj.r_yield)
            self.__total_ore_cost += n_batches*reagent_obj.costs['ORE']
            
        return self.__total_ore_cost

In [25]:
class reagent:
    def __init__(self, reaction_str):
        split_str = reaction_str.split('=>')
        
        rhs = split_str[1].strip().split(' ')
        
        
        self.name = rhs[1]
        self.r_yield = int(rhs[0])
        
        lhs = split_str[0].split(',')
        self.costs = {}
        for term in lhs:
            term = term.strip()
            spl = term.split(' ')
            self.costs[spl[1]] = int(spl[0])
        
        if 'ORE' in self.costs:
            self.is_element = True
        else:
            self.is_element = False
        
        self.__reaction_str = reaction_str
        
    def __repr__(self):                         
        return self.__reaction_str

In [26]:
def reagents_from_reactions(reactions):
    reaction_split = reactions.split('\n')
    reagents = {}
    for reaction_str in reaction_split:
        r = reagent(reaction_str)
        reagents[r.name] = r
    
    return reagents

In [ ]:
test_reactions = '''10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL'''

In [40]:
test_reactions = '''9 ORE => 2 A
8 ORE => 3 B
7 ORE => 5 C
3 A, 4 B => 1 AB
5 B, 7 C => 1 BC
4 C, 1 A => 1 CA
2 AB, 3 BC, 4 CA => 1 FUEL'''

In [49]:
test_reactions = '''157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT'''

In [50]:
reagents = reagents_from_reactions(test_reactions)
reagents

{'NZVS': 157 ORE => 5 NZVS,
 'DCFZ': 165 ORE => 6 DCFZ,
 'FUEL': 44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL,
 'QDVJ': 12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ,
 'PSHF': 179 ORE => 7 PSHF,
 'HKGWZ': 177 ORE => 5 HKGWZ,
 'XJWVT': 7 DCFZ, 7 PSHF => 2 XJWVT,
 'GPVTF': 165 ORE => 2 GPVTF,
 'KHKGT': 3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT}

In [51]:
stash = stockpile(reagents)

In [52]:
stash.order('FUEL', 1)

In [53]:
stash.element_requirements

{'DCFZ': 323, 'PSHF': 366, 'NZVS': 64, 'HKGWZ': 85, 'GPVTF': 10}

In [54]:
stash.make_elements()

24272